Metrike

In [1]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import torch
import torchvision.transforms as T
import torchvision
import cv2
import matplotlib.pyplot as plt

from src.customDataset import *
from src.modelFunkcije import *
from src.metrike import *
from src.utils import *

Ucitavanje modela i dataseta

In [2]:
## ucitavanje modela
model = get_model(num_classes)

# tezine
model.load_state_dict(torch.load('/notebooks/lk-s-2024-detekcija-fudbalera/finetune modeli/frcnn_custom_final.pth'))

# Prevaluacioni režim
model.eval()

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
transform = T.Compose([T.ToTensor()])

# instanca CustomVOCDataset i DataLoader
dataset = CustomVOCDataset(root='/notebooks/lk-s-2024-detekcija-fudbalera/datasetovi/dataset1.zip_sredjen/test', CLASSES=CLASSES, transforms=transform, return_image_id=True)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=4)

c:\Users\Andjela\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Andjela\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FileNotFoundError: [Errno 2] No such file or directory: '/notebooks/lk-s-2024-detekcija-fudbalera/finetune modeli/frcnn_custom_final.pth'

Detekcija 

In [ ]:
# detekcija frcnn na nasem datasetu
predictions = {}

for images, targets, image_ids in data_loader:
    images = list(img.to(device) for img in images)

    with torch.no_grad():
        prediction = model(images)

    for img, pred, image_id in zip(images, prediction, image_ids):
        predictions[image_id] = pred

### groung truth - prave anotacije
annotations = {}
for image_id in dataset.image_ids:
    ann_path = os.path.join(dataset.annotation_dir, f"{image_id}.xml")
    boxes, labels = read_voc_annotations(ann_path, CLASSES)
    annotations[image_id] = {
        'boxes': boxes,
        'labels': labels
    }

Racunanje rezultata

In [ ]:
# Računanje TP, FP, FN metrika
tp, fp, fn = compare_predictions_with_annotations(predictions, annotations, CLASSES, threshold=thrcompare)

# Ispis
for cls_name in CLASSES:
    print(f'Class: {cls_name}')
    print(f'TP: {tp[cls_name]}, FP: {fp[cls_name]}, FN: {fn[cls_name]}')
    print('----------------------')

image_id = dataset.image_ids[0]
img_path = os.path.join(dataset.image_dir, f"{image_id}.jpg")
ann_path = os.path.join(dataset.annotation_dir, f"{image_id}.xml")

img, _ = load_image_and_annotations(img_path, ann_path, CLASSES)
img_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    prediction = model(img_tensor)

plot_image_with_boxes(img_tensor.squeeze(0), prediction[0]['boxes'], prediction[0]['labels'], prediction[0]['scores'], CLASSES)